In [41]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [66]:
from data import Envelope
from system import AccountingSystem
from utils import dollars

In [67]:
sys = AccountingSystem()
sys.load()

In [68]:
import panel as pn

pn.extension("tabulator")
pn.widgets.Tabulator.theme = "materialize"

# Accounts Table

In [69]:
from bokeh.models.widgets.tables import BooleanFormatter, NumberFormatter, SelectEditor

In [70]:
# from panel.viewable import Viewer

# class EditableAccountsTable(Viewer):
#     def __init__(self):
#         self.setup_widgets()

#     def setup_widgets(self):
#         self.add_account_button = pn.widgets.Button(name="Add", button_type="primary")
#         self.accounts_table = pn.widgets.Tabulator(
#             sys.data.accounts,
#             formatters={
#                 "track": BooleanFormatter(),
#                 "amount": NumberFormatter(format="$0,0.00", font_style="bold"),
#             },
#             editors={
#                 "track": SelectEditor(options=["True", "False"]),
#                 "amount": {"type": "number", "verticalNavigation": "table"},
#             },
#         )
#         self.layout = pn.Column(self.accounts_table, self.add_account_button)

In [71]:
add_account_button = pn.widgets.Button(name="Add", button_type="primary")
accounts_table = pn.widgets.Tabulator(
    sys.data.accounts,
    formatters={
        "track": BooleanFormatter(),
        "amount": NumberFormatter(format="$0,0.00", font_style="bold"),
    },
    editors={
        # "track": SelectEditor(options=["True", "False"]),
        'track': None,
        "amount": {"type": "number", "verticalNavigation": "table"},
    },
    selectable = False
)

def check_toggle_track(event):
    if event.column == "track":
        current_value = sys.data.accounts.loc[event.row, "track"]
        sys.data.accounts.loc[event.row, "track"] = not current_value
        accounts_table.value = accounts_table.value = sys.data.accounts

accounts_table.on_click(check_toggle_track)


def add_account(event):
    sys.data.accounts = sys.data.accounts.append(
        dict(name="name", amount=0.0, track=False), ignore_index=True
    )
    accounts_table.value = sys.data.accounts


add_account_button.on_click(add_account)

Watcher(inst=Button(button_type='primary', name='Add'), cls=<class 'panel.widgets.button.Button'>, fn=<function add_account at 0x7f28e3273e20>, mode='args', onlychanged=False, parameter_names=('clicks',), what='value', queued=False, precedence=0)

In [72]:
accounts_editor = pn.Column(accounts_table, add_account_button)
accounts_editor

Column
    [0] Tabulator(editors={'track': None, ...}, formatters={'track': BooleanFormatter...}, selectable=False, value=Empty DataFrame
Columns: [...)
    [1] Button(button_type='primary', name='Add')

In [73]:
type(sys.data.accounts)

pandas.core.frame.DataFrame

In [11]:
# sys.data.accounts.append(
#     {"name": "test", "amount": 1.0, "track": False}, ignore_index=True
# )

In [62]:
sys.data.accounts

,name,amount,track
0,savings,500.0,True


In [63]:
sys.data.envelopes

[Envelope(id=0, name='Unaccounted', category='Internal/Unaccounted', notes='', amount=0.0, goal=None, capped=False, created=None, removed=None, active=True),
 Envelope(id=0, name='Expense Queue', category='Internal/Expense Queue', notes='', amount=0.0, goal=None, capped=False, created=None, removed=None, active=True),
 Envelope(id=0, name='Staged Expense', category='Internal/Staged Expense', notes='', amount=0.0, goal=None, capped=False, created=None, removed=None, active=True)]

In [77]:
sys.take_envelopes_snapshot()
sys.take_accounts_snapshot()

{'Unaccounted': 0.0, 'Expense Queue': 0.0, 'Staged Expense': 0.0, 'date': datetime.datetime(2022, 8, 16, 22, 38, 0, 818520)}
{'checking': 200.0, 'date': datetime.datetime(2022, 8, 16, 22, 38, 0, 821605)}


/home/dwl/lab/panel-accounting-system/system.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.data.envelope_history = self.data.envelope_history.append(new_row, ignore_index=True)
/home/dwl/lab/panel-accounting-system/system.py:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  


In [78]:
sys.data.envelope_history

,Unaccounted,Expense Queue,Staged Expense,date
0,0.0,0.0,0.0,2022-08-16 22:37:05.979075
1,0.0,0.0,0.0,2022-08-16 22:38:00.818520


In [79]:
sys.data.account_history

,checking,date
0,200.0,2022-08-16 22:37:05.983863
1,200.0,2022-08-16 22:38:00.821605


In [13]:
sys.data.accounts.loc[1, "track"] = False

In [ ]:
accounts_editor

In [ ]:
accounts_editor.show()

In [ ]:
def calculate_total():
    return sys.data.accounts[sys.data.accounts.track].amount.sum()


pn.indicators.Number(name="Total", value=calculate_total(), default_color="white")

# Special Envelopes

In [ ]:
import param
from panel.viewable import Viewer

envelope_colors = {
    "Cost": "#992211",
    "Emergency": "#775533",
    "Save": "#228844",
    "Spend": "#224488",
    "Internal/Unaccounted": "#449988",
    "Internal/Expense Queue": "#BB7711",
    "Internal/Staged Expense": "#AA2277",
}


class EnvelopePanel(Viewer):

    css = """
    .bk .envelope-card {
        background: #119933;
        color: white !important;
        border-radius: 20px !important;
        margin: 10px;
        line-height: .5;
    }
    
    .bk .envelope-card h1,
    .bk .envelope-card h3 {
        padding: 0px;
        margin: 0px;
    }
    .bk .envelope-card h3 {
        text-align: left;
        margin-left: -10px;
        margin-top: -5px;
        color: #FFFFFF88;
    }
    .bk .envelope-card small {
        font-size: 12px;
        color: #FFFFFFAA;
    }
    
    .bk .card-button {
        display: none !important;
    }
    
    .bk .curvy {
        border-radius: 20px !important;
    }
    """

    name = param.String(default="str")
    value = param.Number()
    display_graph_in_header = param.Boolean(False)

    def __init__(self, envelope: Envelope):
        self.envelope = envelope
        
        amount = f"<h1>{dollars(envelope.amount)}"
        if envelope.goal is not None:
            amount += f"<small>/{dollars(envelope.goal, dollar_sign=False)}</small"
        amount += "</h1>"
        
        
        super().__init__()
        self.layout = pn.Card(
            pn.pane.Str("hello!"),
            header=pn.FlexBox(
                pn.Column(
                    pn.pane.HTML(
                        f"<h3>{envelope.name}</h3>{amount}"
                    )
                )
            ),
            background=envelope_colors[self.envelope.category],
            header_color="white",
            header_background=envelope_colors[self.envelope.category],
            header_css_classes=["curvy"],
            css_classes=["envelope-card", "curvy"],
        )
        self.layout.collapsed = True

    def update(self):
        """Update visual layout from underlying envelope data."""
        pass

    def __panel__(self):
        return self.layout

In [ ]:
pn.extension("tabulator", raw_css=[EnvelopePanel.css])

In [ ]:
env = Envelope(name="Testing", category="Emergency", amount=123.45)

In [ ]:
envelopes_example = pn.FlexBox(
    EnvelopePanel(Envelope(name="Cost", category="Cost", amount=123.45, goal=200.00, capped=True)),
    EnvelopePanel(Envelope(name="Emergency", category="Emergency", amount=123.45)),
    EnvelopePanel(Envelope(name="Save", category="Save", amount=123.45)),
    EnvelopePanel(Envelope(name="Spend", category="Spend", amount=123.45)),
    EnvelopePanel(
        Envelope(name="Unaccounted", category="Internal/Unaccounted", amount=123.45)
    ),
    EnvelopePanel(
        Envelope(name="Expense Queue", category="Internal/Expense Queue", amount=123.45)
    ),
    EnvelopePanel(
        Envelope(
            name="Staged Expense", category="Internal/Staged Expense", amount=123.45
        )
    ),
)
envelopes_example

In [ ]:
envelopes_example.show()

For normal envelopes, to do transfer:
Select from dropdown other account, then it shows: (note that the default other envelope should be Unaccounted)

    Env1 -> Env2
    amt

and the `->` is clickable to toggle transfer direction.



When click on the internal envelopes, rather than a normal envelope having related transactions etc, only have form related to that envelope:

* Unaccounted: (normal transfer), distribution plan list/button
* Expense Queue: filtered table to unaccounted expenses, so can directly handle them there 
* Staged expense: filtered table to expenses, possibly a clear all as well.